In [1]:
import os
import logging
import re
import gzip
import shutil
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import gffutils

if os.getcwd().endswith('notebook'):
    os.chdir('..')

In [2]:
sns.set(palette='colorblind', font_scale=1.3)
palette = sns.color_palette()
logging.basicConfig(level=logging.INFO, format="%(asctime)s (%(levelname)s) %(message)s")
logger = logging.getLogger(__name__)

## Exploration

### Check how many unique proteomes are available

In [9]:
%%time
proteome_fasta = os.path.join(
    os.getcwd(), 
    'data/bacteria/DB_BACT95_HCLUST0.5/PROTEOMES_FASTA/AnnotatedGenomes/concat_bact95_hclust05.fasta',
)
assembly_re = r'^.+\$(GCA_[^\s]+)\s?.+$'
assembly_set = set()
with open(proteome_fasta) as f:
    for line in f:
        if not line.startswith('>'):
            continue
            
        m = re.match(assembly_re, line)
        if m is None:
            raise ValueError(line)
        
        assembly = m[1]
        if assembly not in assembly_set:
            assembly_set.add(assembly)
        
print(f'{len(assembly_set):,}')

18,343
CPU times: user 2min 30s, sys: 5.62 s, total: 2min 36s
Wall time: 2min 36s


### Open gff

In [43]:
cds_fasta_path = os.path.join(os.getcwd(), 'data/bacteria/GENOMES_FASTA/GCA_900626105.1_PRJEB29220_genomic.fna')
with open(cds_fasta_path) as f:
    cds_dict = SeqIO.to_dict(SeqIO.parse(f, "fasta"))

In [60]:
gff_path = os.path.join(os.getcwd(), 'data/bacteria/GENOMES_FASTA/GCA_900626105.1_PRJEB29220_genomic_prodigal.gff')
annotations = gffutils.create_db(gff_path, ':memory:', merge_strategy='replace')
for i, f in enumerate(annotations.features_of_type('CDS')):
    print(dir(f))
    print(f.seqid, f.id, f.start, f.end, f.strand)
    
    start_ix = start - 1
    end_ix = end
    
    if f.strand == '+':
        seq = cds_dict[f.seqid].seq[start_ix:end_ix]
    else:
        seq = cds_dict[f.seqid].seq.reverse_complement()[start_ix:end_ix]
        
    aa_seq = seq.translate()
    
    print(len(seq))
    print()
    print(seq._data)
    print()
    print(aa_seq)
    print()
    print(len(aa_seq))
    break

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', '__weakref__', 'astuple', 'attributes', 'bin', 'calc_bin', 'chrom', 'dialect', 'end', 'extra', 'featuretype', 'file_order', 'frame', 'id', 'keep_order', 'score', 'seqid', 'sequence', 'sort_attribute_values', 'source', 'start', 'stop', 'strand']
UYZX01000002.1 1_1 299 1567 +
1269

ATGGCTCAGATCAGCCCCATCGTGAAGCAGGCCACCCCCGTGGTCGTAGAGAAGGCCGAGGGCAACTGGATCGTCGGCACCGACGGCGTGCGCTACCTCGACTTCACTTCCGGTATCGGCGTGACCTCGACCGGCCACTGCCACCCGCGCGTCGTCGCGGCCGCCCGCGAGCAGGTCGGCAAAGTCATCCACGCGCAGGCCACGACCGTGATGCACAAGCCGCTGCTCGAGCTCACCGAGAAGCTCAGCGATTACCTGCCCGAGAACCTCGACTCCGTCTTCTACGCGAACTCCGGCGCCGAGGCCGTCGAGGGCGCGTTGCGCCTCGCCCGC

In [67]:
f.attributes.keys()

dict_keys(['ID', 'partial', 'start_type', 'rbs_motif', 'rbs_spacer', 'gc_cont', 'conf', 'score', 'cscore', 'sscore', 'rscore', 'uscore', 'tscore'])

In [73]:
seq_record = SeqRecord(
    seq,
    id=f.id,
    description='description',
)
#SeqIO.write([seq_record], os.path.join(os.getcwd(), 'data/test.fasta'), "fasta")

1

## Unique assemblies? or do we need the version number too

In [78]:
genomes_path = os.path.join(os.getcwd(), 'data/bacteria/GENOMES_FASTA/')
n_records = 0
unique_assemblies = set()
for path in Path(genomes_path).glob('*.fna'):
    m = re.match(r'^(GCA_[^_]+)_.+\.fna$', path.name)
    if m is None:
        print('NULL:', path.name)
    else:
        n_records += 1
        unique_assemblies.add(m[1])

print(f'{n_records:,}', f'{len(unique_assemblies):,}')

18,343 18,343


No need to worry about the version number when naming files